### Huaixi Tang 2631695
### Xinyu Zhang 3056684

# Lab 4: Recurrent models

This lab gives you practice with embeddings (word vectors, in this case) and recurrent neural models in NLP. The first part focuses on embeddings as input to a recurrent model, and the second part focuses on embeddings derived from recurrent models, applying them to the task of word sense disambuiguation following the approach from the original paper by Peters et al.

Everybody's machine is different and the neural computations required for this lab are more demanding than in the other assignments in this course. For this reason, it is advisable to use Google CoLab which guarantees a minimum level of performance. It is also recommended to use GPU acceleration; on CoLab, it can be turned on via <code>Runtime>Change Runtime type>GPU</code>.

## Part 1 (45 points)

In the first part of lab 4, we will play with training a recurrent model for part of speech tagging. As an easy exercise, you will observe what happens when you plug in pretrained word embeddings into a neural NLP model and experiment with different sizes of training data.

If you use Google Colab, it may be easiest to place this notebook and <code>lstm_tutorial.py</code> in <code>/Colab Notebooks</code> directory of your Google Drive. Run the code in the cell just below to enable CoLab to access the files on Drive. This will produce a link to an obscure redirect page which misleadingly suggests that you are getting Google Drive Desktop installation; what happens instead you are allowing the notebook running on Colab access to Drive. Copy the code that appears after you allow Drive access and paste it below.

In [ ]:
#RUN THIS CELL IF USING COLAB TO USE GOOGLE DRIVE FOR STORING lstm_tutorial.py AND/OR DATA FILES
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


The neural network solutions in this lab rely on AllenNLP library version 0.9.0 (the other code of this lab assignment may work incorrectly with more recent versions); <code>overrides</code> is required for compatibility. Linguistic resources are from NLTK version 3.6.2 and might work incorrectly in other versions. Install these before proceeding; installation process may vary depending on your system. On CoLab, this can be done via the following command:

In [ ]:
#IF USING COLAB, INSTALL allennlp AND nltk AS FOLLOWS
!pip install -U allennlp==0.9.0 overrides==3.1.0 nltk==3.6.2

     |████████████████████████████████| 7.6MB 4.3MB/s 
     |████████████████████████████████| 1.5MB 48.5MB/s 
     |████████████████████████████████| 5.6MB 18.4MB/s 
     |████████████████████████████████| 163kB 38.0MB/s 
     |████████████████████████████████| 122kB 50.8MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 133kB 44.5MB/s 
     |████████████████████████████████| 30.8MB 90kB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 266kB 37.5MB/s 
     |████████████████████████████████| 133kB 50.0MB/s 
     |████████████████████████████████| 245kB 28.9MB/s 
     |████████████████████████████████| 256kB 28.2MB/s 
     |████████████████████████████████| 143kB 35.4MB/s 
     |████████████████████████████████| 1.2MB 51.7MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 7.6MB 37.2MB/s 
  

**Before you start**,  import required modules:

In [ ]:
import random
import nltk
import allennlp

If you run this for the first time, you may need to download various data using NLTK:

In [ ]:
nltk.download('brown')
nltk.download('semcor')
nltk.download('wordnet')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Exercise 1: prepare the data (5 points)

Linguistic data come in a variety of formats. You already had a chance to play with POS-annotated corpus data in Lab 1.

In the first exercise, you will access POS-annotated data in one format (NLTK) and save it on the disk in a text format. Start with the tagged sentences from the Brown corpus, which can be retrieved as below:

In [ ]:
nltk.corpus.brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

Now randomize the order of all sentences in the corpus using <code>random.shuffle()</code> function and select the first 50K sentences for training and the next 5K for validation.

In [ ]:
#Write your code here
brown = list(nltk.corpus.brown.tagged_sents())
random.shuffle(brown)

training_brown=brown[:50000]
validation_brown=brown[50000:55000]
testing_brown=brown[55000:]

Define a function for saving your datasets to a text file in the following format:
* one sentence per line
* tokens separated by spaces
* POS tag separated from the token by "###", for example <code>said###VBD</code>.

In [ ]:
def write_posdata(sentences,outfile):
    #Write your code here
    with open(outfile,'w') as f:
      for sentence in sentences:
        tokens = [token[0]+'###'+token[1]+' ' for token in sentence]
        f.writelines(tokens)
        f.write('\n')
      f.close()

Now save your data partitions in different sizes. We will start with small data samples since training on a large dataset may be very slow depending on your machine. We won't use the full 50K sentence training set in this lab since this might take too long.

In [ ]:
write_posdata(training_brown,"train_brown.txt")
write_posdata(validation_brown,"validation_brown.txt")
write_posdata(training_brown[:50],"train_brown_50.txt")
write_posdata(validation_brown[:50],"validation_brown_50.txt")
write_posdata(training_brown[:500],"train_brown_500.txt")
write_posdata(validation_brown[:500],"validation_brown_500.txt")
write_posdata(training_brown[:5000],"train_brown_5000.txt")
write_posdata(validation_brown[:5000],"validation_brown_5000.txt")

Congratulations, you have now saved the POS tagged data for model training purposes!

## Exercise 2: train neural POS tagger models (15 points)

We will now play with a neural model. You have installed <code>allennlp</code> which contains all necessary components for this and the training code for an LSTM model, which follows an old AllenNLP tutorial, is contained in <code>lstm_tutorial.py</code>. PLace the latter in the same directory as this notebook. Let us start by loading the model code and data, starting with a tiny sample for demonstration purposes. 

In [ ]:
from lstm_tutorial import *

train_dataset_tiny = reader.read("train_brown_50.txt")
validation_dataset_tiny = reader.read("validation_brown_50.txt")

50it [00:00, 7245.30it/s]
50it [00:00, 8448.76it/s]


Fist of all we need to initialize the vocabulary and define an embedding (vector) for each token. We set the embedding size at 300, common in realistic applications. By default, the embeddings are initialized randomly and updated during trining (this can be changed but we start with a standard configuration). We also need to specify the <code>HIDDEN_DIM</code> parameter: the dimensionality of the hidden vector representations in the LSTM cell.

In [ ]:
vocab_tiny = Vocabulary.from_instances(train_dataset_tiny + validation_dataset_tiny)

EMBEDDING_DIM = 300
HIDDEN_DIM = 20

token_embedding_tiny = Embedding(num_embeddings=vocab_tiny.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

100%|██████████| 100/100 [00:00<00:00, 21287.64it/s]


Download the smallest pretrained word vector model from https://nlp.stanford.edu/projects/glove/, unzip it, and extract the relevant file <code>'glove.6B.300d.txt'</code> in your working directory. The size of the file is 1GB; if using Google Drive with Colab, make sure you have sufficient space. Downloading and uploading the file might take a few minutes. You can <b>either</b> upload the relevant file from your personal machine `or` use the code below directly from CoLab:

In [ ]:
#THIS CELL IS OPTIONAL, TO BE USED ON COLAB. YOU CAN USE wget AS BELOW OR ALTERNATIVELY UPLOAD GloVe EMBEDDINGS TO GOOGLE DRIVE FROM YOUR MACHINE
# download the file
!wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip the file
!unzip -d . 'glove.6B.zip'
# remove useless contents
!rm 'glove.6B.200d.txt' 'glove.6B.100d.txt' 'glove.6B.50d.txt'

--2021-06-21 14:14:29--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-06-21 14:14:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-06-21 14:14:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.2’

glove

Initialize token embeddings with values from pretrained GloVe model:


In [ ]:
glove_token_embedding_tiny = Embedding.from_params(vocab=vocab_tiny,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))

400000it [00:02, 177421.83it/s]


Now from embedding a single word with <code>token_embedding_tiny</code> we can proceed to mapping a word sequence into a sequence of vectors:

In [ ]:
word_embeddings_tiny = BasicTextFieldEmbedder({"tokens": token_embedding_tiny})

The following initializes parameters of an LSTM model using <code>word_embeddings_tiny</code> input encoding

In [ ]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model_tiny = LstmTagger(word_embeddings_tiny, lstm, vocab_tiny)

Now define an LSTM model called <code>glove_model_tiny</code> that uses <code>glove_token_embedding_tiny</code>:

In [ ]:
#write your code here
glove_word_embeddings_tiny = BasicTextFieldEmbedder({"tokens": glove_token_embedding_tiny})
glove_lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
glove_model_tiny = LstmTagger(glove_word_embeddings_tiny, glove_lstm, vocab_tiny)

Train the basic model for the tiny dataset. **Do not clear the output of this cell in the submitted version.**



In [ ]:
basic_trainer_tiny=initialize_trainer(model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
basic_trainer_tiny.train()

accuracy: 0.0331, loss: 4.3551 ||: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]
accuracy: 0.0596, loss: 4.3522 ||: 100%|██████████| 1/1 [00:00<00:00, 77.45it/s]
accuracy: 0.0491, loss: 4.3492 ||: 100%|██████████| 1/1 [00:00<00:00, 56.56it/s]
accuracy: 0.0928, loss: 4.3467 ||: 100%|██████████| 1/1 [00:00<00:00, 48.35it/s]
accuracy: 0.0832, loss: 4.3434 ||: 100%|██████████| 1/1 [00:00<00:00, 39.14it/s]
accuracy: 0.1116, loss: 4.3413 ||: 100%|██████████| 1/1 [00:00<00:00, 89.99it/s]
accuracy: 0.1033, loss: 4.3376 ||: 100%|██████████| 1/1 [00:00<00:00, 52.31it/s]
accuracy: 0.1210, loss: 4.3358 ||: 100%|██████████| 1/1 [00:00<00:00, 83.45it/s]
accuracy: 0.1093, loss: 4.3317 ||: 100%|██████████| 1/1 [00:00<00:00, 56.31it/s]
accuracy: 0.1218, loss: 4.3304 ||: 100%|██████████| 1/1 [00:00<00:00, 57.35it/s]
accuracy: 0.1103, loss: 4.3259 ||: 100%|██████████| 1/1 [00:00<00:00, 44.26it/s]
accuracy: 0.1218, loss: 4.3250 ||: 100%|██████████| 1/1 [00:00<00:00, 61.00it/s]
accuracy: 0.1103, loss: 4.32

{'best_epoch': 999,
 'best_validation_accuracy': 0.4131175468483816,
 'best_validation_loss': 2.6033623218536377,
 'epoch': 999,
 'peak_cpu_memory_MB': 3544.792,
 'peak_gpu_0_memory_MB': 1130,
 'training_accuracy': 0.4282848545636911,
 'training_cpu_memory_MB': 3544.792,
 'training_duration': '0:02:18.103302',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 1130,
 'training_loss': 2.4295542240142822,
 'training_start_epoch': 0,
 'validation_accuracy': 0.4131175468483816,
 'validation_loss': 2.6033623218536377}

You have trained an LSTM POS tagger for the basic model. Now train the <code>glove_model_tiny</code>. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#Write your code here
glove_trainer_tiny=initialize_trainer(glove_model_tiny,vocab_tiny,train_dataset_tiny,validation_dataset_tiny,batch_size=50)
glove_trainer_tiny.train()

accuracy: 0.0181, loss: 4.4113 ||: 100%|██████████| 1/1 [00:00<00:00, 53.25it/s]
accuracy: 0.0119, loss: 4.4083 ||: 100%|██████████| 1/1 [00:00<00:00, 84.66it/s]
accuracy: 0.0211, loss: 4.4035 ||: 100%|██████████| 1/1 [00:00<00:00, 64.02it/s]
accuracy: 0.0179, loss: 4.4011 ||: 100%|██████████| 1/1 [00:00<00:00, 71.97it/s]
accuracy: 0.0241, loss: 4.3958 ||: 100%|██████████| 1/1 [00:00<00:00, 52.75it/s]
accuracy: 0.0239, loss: 4.3939 ||: 100%|██████████| 1/1 [00:00<00:00, 78.93it/s]
accuracy: 0.0281, loss: 4.3880 ||: 100%|██████████| 1/1 [00:00<00:00, 55.65it/s]
accuracy: 0.0290, loss: 4.3867 ||: 100%|██████████| 1/1 [00:00<00:00, 117.49it/s]
accuracy: 0.0381, loss: 4.3803 ||: 100%|██████████| 1/1 [00:00<00:00, 49.34it/s]
accuracy: 0.0349, loss: 4.3795 ||: 100%|██████████| 1/1 [00:00<00:00, 80.20it/s]
accuracy: 0.0491, loss: 4.3726 ||: 100%|██████████| 1/1 [00:00<00:00, 55.69it/s]
accuracy: 0.0426, loss: 4.3724 ||: 100%|██████████| 1/1 [00:00<00:00, 98.03it/s]
accuracy: 0.0592, loss: 4.3

{'best_epoch': 999,
 'best_validation_accuracy': 0.520442930153322,
 'best_validation_loss': 2.0678558349609375,
 'epoch': 999,
 'peak_cpu_memory_MB': 3544.988,
 'peak_gpu_0_memory_MB': 1130,
 'training_accuracy': 0.59679037111334,
 'training_cpu_memory_MB': 3544.988,
 'training_duration': '0:02:16.670917',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 1130,
 'training_loss': 1.629144549369812,
 'training_start_epoch': 0,
 'validation_accuracy': 0.520442930153322,
 'validation_loss': 2.0678558349609375}

## Exercise 3: Explore training parameters (25 points)

Create separate models on the basis of bigger datasets: the 500 sentence training and 500 sentence validation and 5000 sentence training and 5000 sentence validation. Using the full training set (50K sentences) is optional (your machine might be too slow). Initialize and train the basic model on 500 sentence training and 500 sentence validation data. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#train the basic model on 500 sentences
train_dataset_500 = reader.read("train_brown_500.txt")
validation_dataset_500 = reader.read("validation_brown_500.txt")

vocab_500 = Vocabulary.from_instances(train_dataset_500 + validation_dataset_500)

token_embedding_500 = Embedding(num_embeddings=vocab_500.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

word_embeddings_500 = BasicTextFieldEmbedder({"tokens": token_embedding_500})

lstm_500 = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model_500 = LstmTagger(word_embeddings_500, lstm_500, vocab_500)

basic_trainer_500=initialize_trainer(model_500,vocab_500,train_dataset_500,validation_dataset_500,batch_size=50)
basic_trainer_500.train()

500it [00:00, 11884.51it/s]
500it [00:00, 17296.38it/s]
100%|██████████| 1000/1000 [00:00<00:00, 57090.99it/s]
accuracy: 0.0022, loss: 5.1287 ||: 100%|██████████| 10/10 [00:00<00:00, 93.87it/s]
accuracy: 0.0012, loss: 5.0943 ||: 100%|██████████| 10/10 [00:00<00:00, 133.21it/s]
accuracy: 0.0018, loss: 5.0694 ||: 100%|██████████| 10/10 [00:00<00:00, 112.52it/s]
accuracy: 0.0232, loss: 5.0346 ||: 100%|██████████| 10/10 [00:00<00:00, 172.94it/s]
accuracy: 0.1151, loss: 5.0119 ||: 100%|██████████| 10/10 [00:00<00:00, 107.97it/s]
accuracy: 0.1346, loss: 4.9758 ||: 100%|██████████| 10/10 [00:00<00:00, 169.68it/s]
accuracy: 0.1311, loss: 4.9547 ||: 100%|██████████| 10/10 [00:00<00:00, 110.36it/s]
accuracy: 0.1346, loss: 4.9167 ||: 100%|██████████| 10/10 [00:00<00:00, 173.05it/s]
accuracy: 0.1311, loss: 4.8967 ||: 100%|██████████| 10/10 [00:00<00:00, 114.49it/s]
accuracy: 0.1346, loss: 4.8560 ||: 100%|██████████| 10/10 [00:00<00:00, 158.63it/s]
accuracy: 0.1311, loss: 4.8367 ||: 100%|██████████

{'best_epoch': 972,
 'best_validation_accuracy': 0.7234766178554558,
 'best_validation_loss': 1.2675086855888367,
 'epoch': 981,
 'peak_cpu_memory_MB': 3552.824,
 'peak_gpu_0_memory_MB': 1172,
 'training_accuracy': 0.91141049193713,
 'training_cpu_memory_MB': 3552.824,
 'training_duration': '0:04:21.690277',
 'training_epochs': 981,
 'training_gpu_0_memory_MB': 1172,
 'training_loss': 0.5280864983797073,
 'training_start_epoch': 0,
 'validation_accuracy': 0.7239489844119036,
 'validation_loss': 1.2683733224868774}

Now do the same training (500 sentence training and 500 sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#write your code here
glove_token_embedding_500 = Embedding.from_params(vocab=vocab_500,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))
glove_word_embeddings_500 = BasicTextFieldEmbedder({"tokens": glove_token_embedding_500})

glove_lstm_500 = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

glove_model_500 = LstmTagger(glove_word_embeddings_500, glove_lstm_500, vocab_500)

glove_trainer_500=initialize_trainer(glove_model_500,vocab_500,train_dataset_500,validation_dataset_500,batch_size=50)
glove_trainer_500.train()

400000it [00:02, 144954.63it/s]
accuracy: 0.0097, loss: 5.0840 ||: 100%|██████████| 10/10 [00:00<00:00, 111.09it/s]
accuracy: 0.0430, loss: 5.0396 ||: 100%|██████████| 10/10 [00:00<00:00, 170.58it/s]
accuracy: 0.0740, loss: 5.0112 ||: 100%|██████████| 10/10 [00:00<00:00, 111.53it/s]
accuracy: 0.1160, loss: 4.9647 ||: 100%|██████████| 10/10 [00:00<00:00, 161.49it/s]
accuracy: 0.1230, loss: 4.9360 ||: 100%|██████████| 10/10 [00:00<00:00, 108.51it/s]
accuracy: 0.1337, loss: 4.8852 ||: 100%|██████████| 10/10 [00:00<00:00, 155.74it/s]
accuracy: 0.1305, loss: 4.8548 ||: 100%|██████████| 10/10 [00:00<00:00, 108.82it/s]
accuracy: 0.1345, loss: 4.7980 ||: 100%|██████████| 10/10 [00:00<00:00, 163.35it/s]
accuracy: 0.1310, loss: 4.7650 ||: 100%|██████████| 10/10 [00:00<00:00, 101.59it/s]
accuracy: 0.1346, loss: 4.7008 ||: 100%|██████████| 10/10 [00:00<00:00, 159.21it/s]
accuracy: 0.1311, loss: 4.6648 ||: 100%|██████████| 10/10 [00:00<00:00, 111.12it/s]
accuracy: 0.1346, loss: 4.5929 ||: 100%|████

{'best_epoch': 999,
 'best_validation_accuracy': 0.7764761454888994,
 'best_validation_loss': 1.0784356594085693,
 'epoch': 999,
 'peak_cpu_memory_MB': 3565.232,
 'peak_gpu_0_memory_MB': 1172,
 'training_accuracy': 0.908858950806287,
 'training_cpu_memory_MB': 3565.232,
 'training_duration': '0:04:27.134712',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 1172,
 'training_loss': 0.41714711785316466,
 'training_start_epoch': 0,
 'validation_accuracy': 0.7764761454888994,
 'validation_loss': 1.0784356594085693}

Use a bigger training set now with 5K sentence training and 5K sentence validation sets and random initial embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#write your code here
train_dataset_5000 = reader.read("train_brown_5000.txt")
validation_dataset_5000 = reader.read("validation_brown_5000.txt")

vocab_5000 = Vocabulary.from_instances(train_dataset_5000 + validation_dataset_5000)

token_embedding_5000 = Embedding(num_embeddings=vocab_5000.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

word_embeddings_5000 = BasicTextFieldEmbedder({"tokens": token_embedding_5000})

lstm_5000 = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model_5000 = LstmTagger(word_embeddings_5000, lstm_5000, vocab_5000)

basic_trainer_5000=initialize_trainer(model_5000,vocab_5000,train_dataset_5000,validation_dataset_5000,batch_size=50)
basic_trainer_5000.train()

5000it [00:00, 8073.46it/s]
5000it [00:00, 22754.40it/s]
100%|██████████| 10000/10000 [00:00<00:00, 46998.53it/s]
accuracy: 0.1016, loss: 5.4767 ||: 100%|██████████| 100/100 [00:01<00:00, 90.07it/s]
accuracy: 0.1323, loss: 5.1572 ||: 100%|██████████| 100/100 [00:00<00:00, 125.56it/s]
accuracy: 0.1302, loss: 4.7145 ||: 100%|██████████| 100/100 [00:00<00:00, 108.63it/s]
accuracy: 0.1323, loss: 4.2887 ||: 100%|██████████| 100/100 [00:00<00:00, 166.80it/s]
accuracy: 0.1311, loss: 4.1041 ||: 100%|██████████| 100/100 [00:00<00:00, 110.07it/s]
accuracy: 0.1323, loss: 3.9645 ||: 100%|██████████| 100/100 [00:00<00:00, 165.58it/s]
accuracy: 0.1334, loss: 3.8953 ||: 100%|██████████| 100/100 [00:01<00:00, 70.98it/s]
accuracy: 0.1323, loss: 3.8314 ||: 100%|██████████| 100/100 [00:00<00:00, 166.65it/s]
accuracy: 0.1347, loss: 3.7931 ||: 100%|██████████| 100/100 [00:00<00:00, 110.09it/s]
accuracy: 0.1331, loss: 3.7461 ||: 100%|██████████| 100/100 [00:00<00:00, 165.50it/s]
accuracy: 0.1500, loss: 3.70

{'best_epoch': 282,
 'best_validation_accuracy': 0.8524937226133622,
 'best_validation_loss': 0.7564672347903252,
 'epoch': 291,
 'peak_cpu_memory_MB': 4388.248,
 'peak_gpu_0_memory_MB': 1292,
 'training_accuracy': 0.937659334795755,
 'training_cpu_memory_MB': 4388.248,
 'training_duration': '0:08:09.546544',
 'training_epochs': 291,
 'training_gpu_0_memory_MB': 1292,
 'training_loss': 0.3256991972029209,
 'training_start_epoch': 0,
 'validation_accuracy': 0.85244448820836,
 'validation_loss': 0.7609185263514519}

Now do the same training (5K sentence training and 5K sentence validation sets) with GloVE embeddings. **Do not clear the output of this cell in the submitted version.**

In [ ]:
#write your code here
glove_token_embedding_5000 = Embedding.from_params(vocab=vocab_5000,
                            params=Params({'pretrained_file':'glove.6B.300d.txt',
                                           'embedding_dim' : EMBEDDING_DIM}))
glove_word_embeddings_5000 = BasicTextFieldEmbedder({"tokens": glove_token_embedding_5000})

glove_lstm_5000 = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

glove_model_5000 = LstmTagger(glove_word_embeddings_5000, glove_lstm_5000, vocab_5000)

glove_trainer_5000 = initialize_trainer(glove_model_5000,vocab_5000,train_dataset_5000,validation_dataset_5000,batch_size=50)
glove_trainer_5000.train()

400000it [00:03, 116234.91it/s]
accuracy: 0.1104, loss: 5.2590 ||: 100%|██████████| 100/100 [00:00<00:00, 108.48it/s]
accuracy: 0.1323, loss: 4.5516 ||: 100%|██████████| 100/100 [00:00<00:00, 160.77it/s]
accuracy: 0.1422, loss: 4.2133 ||: 100%|██████████| 100/100 [00:00<00:00, 109.46it/s]
accuracy: 0.1640, loss: 3.9855 ||: 100%|██████████| 100/100 [00:00<00:00, 164.37it/s]
accuracy: 0.1995, loss: 3.8624 ||: 100%|██████████| 100/100 [00:00<00:00, 109.80it/s]
accuracy: 0.2622, loss: 3.7399 ||: 100%|██████████| 100/100 [00:00<00:00, 164.67it/s]
accuracy: 0.2743, loss: 3.6544 ||: 100%|██████████| 100/100 [00:00<00:00, 108.67it/s]
accuracy: 0.2492, loss: 3.5621 ||: 100%|██████████| 100/100 [00:00<00:00, 163.53it/s]
accuracy: 0.3046, loss: 3.4705 ||: 100%|██████████| 100/100 [00:00<00:00, 111.35it/s]
accuracy: 0.3510, loss: 3.3683 ||: 100%|██████████| 100/100 [00:00<00:00, 162.41it/s]
accuracy: 0.3632, loss: 3.2836 ||: 100%|██████████| 100/100 [00:00<00:00, 110.22it/s]
accuracy: 0.3822, loss

{'best_epoch': 364,
 'best_validation_accuracy': 0.867067106494018,
 'best_validation_loss': 0.6844589185714721,
 'epoch': 373,
 'peak_cpu_memory_MB': 4408.524,
 'peak_gpu_0_memory_MB': 1350,
 'training_accuracy': 0.909922728799826,
 'training_cpu_memory_MB': 4408.524,
 'training_duration': '0:10:36.767377',
 'training_epochs': 373,
 'training_gpu_0_memory_MB': 1350,
 'training_loss': 0.36411905005574224,
 'training_start_epoch': 0,
 'validation_accuracy': 0.868199497809069,
 'validation_loss': 0.686313865184784}

For each trained model, record validation accuracy and training duration (they are returned along with other training stats after training a model) and accuracy on the training set. Fill in the numbers in the table below:

| model | validation accuracy | training accuracy | training duration|
|-------|---------------------|---------------|-------------------------------------------
| basic model on 50 sentences|0.4131|0.4283|0:02:18.103302|
| glove model on 50 sentences|0.5204|0.5968|0:02:16.670917|
| basic model on 500 sentences|0.7239|0.9114|0:04:21.690277|
| glove model on 500 sentences|0.7765|0.9089|0:04:27.134712|
| basic model on 5000 sentences|0.8524|0.9377|0:08:09.546544|
| glove model on 5000 sentences|0.8682|0.9099|0:10:36.767377|

**Question.** What do you conclude from these comparisons? when can it be especially beneficial to initialize a model with pretrained embeddings?

**Answer.** We can see that for each of 50,500,5000 training set, validation acc of glove is higher than the basic model. And the larger the dataset is, the higher the validation and training accuracy is, the longer the training duration is.<br>
When the size of training set is limited, pre-trained embeddings will especially benefit the training model. When the training set is comparatively large, such benefit will not be that obvious while still exist(validation acc of glove is always higher).

## Comment 
In this lab we used pretrained GloVe embeddings in a model for part of speech tagging. GloVe in its turn is also a neural word embedding model, but it had been trained on a completely different objective. GloVe vectors had been optimised on word cooccurrence matrix decomposition, i.e. on the task of predicting which words tend to occur with which other words. Part of speech certainly plays a role in determining statistical cooccurrence of words, but this role is indirect, and explicit part of speech information has not been used in training GloVe.

This makes our application an example of **transfer learning**, whereby a learned model trained on one objective (e.g. word cooccurrence) can benefit a different application (e.g. POS tagging), because some information is shared between them. 

## Part 2 - ELMo vectors (55 points)

> Indented block



In the second part of this lab we will reproduce the word sense disambiguation strategy that the authors of the ELMo vectors explored. The strategy consists in the following:

- create ELMo embeddings for all tokens in a sense-annotated corpus
- calculate mean sense vectors for each word sense in the training partition of the corpus
- for each sense-annotated token in the test partition of the corpus, assign it to the sense of the word to which its ELMo vector is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word by default.

As a sense annotated corpus, we can use SemCor, conveniently available within NLTK. <code>semcor.sents()</code> iterates over all sentences represented as lists of tokens, while <code>semcor.tagged_sents()</code> iterates over the same sentences with additional annotation including WordNet lemma identifiers (lemmas in WordNet stand for a word taken in a specific sense).

In [ ]:
from nltk.corpus import wordnet as wn  
from nltk.corpus import semcor

semcor.sents()
semcor.tagged_sents(tag="sem")

[[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']], [['The'], Tree(Lemma('jury.n.01.jury'), ['jury']), Tree(Lemma('far.r.02.far'), ['further']), Tree(Lemma('state.v.01.say'), ['said']), ['in'], Tree(Lemma('term.n.02.term'), ['term']), Tree(Lemma('end.n.02.end'), ['end']), Tree(Lemma('presentment.n.01.presentment'), ['presentments']), ['

## Exercise 1. Extract relevant data from SemCor (5 points)

Let's prepare SemCor data for the disambiguation task. Since this is just an educational exercise and we don't aim at replicating the full results, we can use only a subset of SemCor. Take the first 10K sentences of SemCor and split them randomly into 90% training and 10% testing partitions:

In [ ]:
#write your code here
semcor_sents_10k=list(semcor.sents()[:10000])
semcor_tag_10k=list(semcor.tagged_sents(tag="sem")[:10000])

r=random.random

random.seed(2)
random.shuffle(semcor_sents_10k,random=r)

random.seed(2)
random.shuffle(semcor_tag_10k,random=r)

semcor_train=semcor_sents_10k[:9000]
semcor_tag_train=semcor_tag_10k[:9000]

semcor_test=semcor_sents_10k[9000:]
semcor_tag_test=semcor_tag_10k[9000:]

Create a function that takes as input a sentence from SemCor and extracts a list which contains, for each token of the sentence, either the corresponding WordNet Lemma (e.g. <code>Lemma('friday.n.01.Friday')</code>) or <code>None</code>. <code>None</code> corresponds to tokens that are either 1) not annotated for word senses (e.g. articles); 2) are marked up as (part of) a named entity (e.g. "City of Atlanta" or placename "Fulton" annotated as  <code>Tree(Lemma('location.n.01.location'), [Tree('NE', ['Fulton'])])</code>). For example, <code>get_lemmas</code> for the first sentence of Semcor should return <code>[None,
 None,
 None,
 None,
 None,
 Lemma('state.v.01.say'),
 Lemma('friday.n.01.Friday'),
 None,
 Lemma('probe.n.01.investigation'),
 None,
 Lemma('atlanta.n.01.Atlanta'),
 None,
 Lemma('late.s.03.recent'),
 Lemma('primary.n.01.primary_election'),
 Lemma('primary.n.01.primary_election'),
 Lemma('produce.v.04.produce'),
 None,
 None,
 Lemma('evidence.n.01.evidence'),
 None,
 None,
 None,
 Lemma('abnormality.n.04.irregularity'),
 Lemma('happen.v.01.take_place'),
 Lemma('happen.v.01.take_place'),
 None]</code>

In [ ]:
def get_lemmas(semcor_sentence):
    #your code here
    extracted_list=[]
    for token in semcor_sentence:
        if type(token)==list:
            for t in token:
                extracted_list.append(None)
        elif type(token[0])==nltk.tree.Tree:
            for i in range(len(token[0])):
                extracted_list.append(None)
        else:
            # print(token,":",len(token))
            for i in range(len(token)):
                extracted_list.append(token.label())

    return extracted_list

In [ ]:
get_lemmas(semcor.tagged_sents(tag="sem")[0])

[None,
 None,
 None,
 None,
 None,
 Lemma('state.v.01.say'),
 Lemma('friday.n.01.Friday'),
 None,
 Lemma('probe.n.01.investigation'),
 None,
 Lemma('atlanta.n.01.Atlanta'),
 None,
 Lemma('late.s.03.recent'),
 Lemma('primary.n.01.primary_election'),
 Lemma('primary.n.01.primary_election'),
 Lemma('produce.v.04.produce'),
 None,
 None,
 Lemma('evidence.n.01.evidence'),
 None,
 None,
 None,
 Lemma('abnormality.n.04.irregularity'),
 Lemma('happen.v.01.take_place'),
 Lemma('happen.v.01.take_place'),
 None]

You are now able to extract word senses (instantiated by WordNet lemmas) from the corpus. The next step is to associate senses with ELMo vectors. Create a dictionary of contextualized token embeddings from the training corpus grouped by the WordNet sense:

In [ ]:
from collections import defaultdict

Train_embeddings=defaultdict(list)

Now let's create contextualized ELMo word embeddings for the tokens in this corpus. We can load the pretrained ELMo model and define a function <code>sentences_to_elmo()</code> that receives a list of tokenized sentences as input and produces their ELMo vectors.

In [ ]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0)

def sentences_to_elmo(sentences):
    character_ids = batch_to_ids(sentences)
    embeddings = elmo(character_ids)
    return embeddings

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Now you can process the corpus sentences and produce their ELMo vectors. It is recommended to pass the input to ELMo encoder in batches. A suggested batch size is 50 sentences. For example, the code below processes the first 50 sentences from the corpus:

In [ ]:
sentences=semcor.sents()[:50]

embeddings=sentences_to_elmo(sentences)

The <code>embeddings</code> that we obtained is a dictionary that contains a list of ELMo embeddings and a list of masks. The mask tells us which embeddings correspond to tokens in the original input sentences and which correspond to the padding (introduced to give all sentences in the batch the same length).
In principle all embeddings are stored in PyTorch tensors so that they can be used in bigger neural models, but we are not going to do it now.

In [ ]:
embeddings['elmo_representations'][0]

tensor([[[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -1.1715e-02,
           7.0427e-02, -4.1873e-01],
         [-3.7781e-01,  2.8141e-01, -2.5836e-01,  ..., -4.8547e-01,
           2.5508e-01,  3.6381e-02],
         [ 9.1191e-01,  1.1779e+00, -8.4833e-01,  ...,  9.8472e-01,
           3.3675e-01,  1.6172e-01],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-6.4618e-03,  6.0215e-03, -3.5598e-01,  ..., -4.4876e-02,
           1.1313e-01, -9.9628e-02],
         [ 1.3721e-01, -2.0003e-01, -1.3074e-01,  ...,  5.9482e-01,
           9.3387e-01, -2.6757e-01],
         [ 1.7280e-01,  1.0801e+00, -5.4539e-01,  ...,  3.1966e-01,
          -5.6408e-01,  3.2461e-01],
         ...,
         [ 0.0000e+00,  0

We can check the size of the embeddings we got. It has three dimensions: 1) the number of sentences 2) the number of tokens (corresponds to the tokens in the longest original sentence of the batch; shorter ones were padded) and 3) the dimensionality of the Elmo vector (1024).

In [ ]:
embeddings['elmo_representations'][0].detach().size()

torch.Size([50, 59, 1024])

Another thing contained in the <code>embeddings</code> is the mask, a tensor encoding which token vectors correspond to original tokens and which are paddings. It has two dimensions, one corresponding to the sentences in the batch (50) and one corresponding to the token positions:

In [ ]:
print(embeddings['mask'].size())
embeddings['mask']

torch.Size([50, 59])


tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

## Exercise 2. Extract ELMo encoding of sentences using a mask (5 points)  

Now define a function <code>get_masked_vectors(embeddings)</code> that takes embeddings as input and returns a list of ELMo sentence encodings to which the mask has been applied.  The output should be a list of Torch tensors, where the padding vectors have been removed so each sentence is represented by an nx1024 tensor where n is sentence length.

In [ ]:
def get_masked_vectors(embeddings):
    #Your code here
    elmo = embeddings['elmo_representations'][0]
    masks = embeddings['mask'].unsqueeze(2)
    outVec = elmo * masks
    
    outVec = list(torch.chunk(outVec,outVec.size()[0],dim=0))
    for i, out in enumerate(outVec):
      out = torch.squeeze(out,dim=0)
      out = list(torch.chunk(out,out.size()[0],dim=0))
      out = [token for token in out if torch.sum(token)!=0]
      out = torch.cat(out,dim=0)
      outVec[i] = out

    return outVec

## Exercise 3. Collect ELMo vectors from the training corpus (20 points)

Process the corpus updating your train word sense vectors in the dictionary. Iterate over the all the train sentences in the corpus, and retrieve for each lemma-annotated token (where lemma is not <code>None</code>) the corresponding ELMo vector. Store the ELMo sense embeddings that correspond to each lemma in the dictionary <code>Train_embeddings</code>. This step of processing the training corpus with ELMo is the most time consuming part of this assignment. However, it should not take forever. If this computation takes more than an hour, you may want to optimize your code or make sure you are using GPU acceleration. For the purposes of developing and debugging your solution, you may start by use a sample of 100 sentences, but then switch to the full 9K sentence training set. 

In [2]:
import torch
#Your code here
def batch_get_lemmas(embeddings_list, sentences_tag, batch_size):
    for i in range(batch_size):
        lemmas_list=get_lemmas(sentences_tag[i])
        for k in range(len(lemmas_list)):
            if not lemmas_list[k]:
                continue
            #add after list or add into default empty list
            Train_embeddings[lemmas_list[k]].append(embeddings_list[i][k])

batch_size=50
#### Due to memory constraints, we can only train on half of the data ####
for bat in range(int(len(semcor_train)/batch_size/2)):
    embeddings=sentences_to_elmo(semcor_train[bat*batch_size:(bat+1)*batch_size])

    print("###batch No. ",bat+1,"###")
    batch_get_lemmas(embeddings['elmo_representations'][0], semcor_tag_train[bat*batch_size:(bat+1)*batch_size], batch_size)

###batch No.  1 ###
###batch No.  2 ###
###batch No.  3 ###
###batch No.  4 ###
###batch No.  5 ###
###batch No.  6 ###
###batch No.  7 ###
###batch No.  8 ###
###batch No.  9 ###
###batch No.  10 ###
###batch No.  11 ###
###batch No.  12 ###
###batch No.  13 ###
###batch No.  14 ###
###batch No.  15 ###
###batch No.  16 ###
###batch No.  17 ###
###batch No.  18 ###
###batch No.  19 ###
###batch No.  20 ###
###batch No.  21 ###
###batch No.  22 ###
###batch No.  23 ###
###batch No.  24 ###
###batch No.  25 ###
###batch No.  26 ###
###batch No.  27 ###
###batch No.  28 ###
###batch No.  29 ###
###batch No.  30 ###
###batch No.  31 ###
###batch No.  32 ###
###batch No.  33 ###
###batch No.  34 ###
###batch No.  35 ###
###batch No.  36 ###
###batch No.  37 ###
###batch No.  38 ###
###batch No.  39 ###
###batch No.  40 ###
###batch No.  41 ###
###batch No.  42 ###
###batch No.  43 ###
###batch No.  44 ###
###batch No.  45 ###
###batch No.  46 ###
###batch No.  47 ###
###batch No.  48 ###
#

How many senses does your Train_embeddings contain? **Do not clear the output of this cell in your submission**

In [10]:
print(len(Train_embeddings))

10051


## Exercise 4. Vector averaging (5 points)

Your <code>Train_embeddings</code> now is a list of all vectors for a given word sense in the training corpus. For our purposes, we do not need the full list but the mean vector for each sense. For each sense in <code>Train_embeddings</code>, substitute the list by the average ELMo vector on the list. One efficient way to do this is to convert the list to a tensor via <code>stack</code> function and use Torch's <code>mean</code> function. Below is an example of how an average of two (random) vectors stored in a tensor can be computed in PyTorch: 

In [ ]:
 randtensor= torch.randn(2, 4)
 print("Tensor storing two 4-dimensional vectors:\n",randtensor)
 print("Average vector: \n",randtensor.mean(dim=0))

Now you are ready to update your <code>Train_embeddings</code> so that it maps lemmas not to lists but to averaged vectors.

In [ ]:
for key, value in Train_embeddings.items():
    vec = torch.stack(value,dim=0)
    vec = torch.mean(vec,dim=0)
    assert (vec.size()==torch.Size([1024]))
    Train_embeddings[key] = vec

## Exercise 5. Testing the sense vectors (20 points)

Test your sense embeddings on your test data, which is a subset of the SemCor corpus. Use the strategy outlined above, with 1st WordNet sense as a fallback: 

- rely on mean sense vectors for each word sense in the training partition of the corpus, as stored in <code>Train_embeddings</code>
- for each sense-annotated token <i>t</i> (e.g. the verb "run") in the test partition of the corpus, assign it to the sense of the word "Lemma('X.v.n.run')" to which ithe ELMo vector <i>t</i> is the closest according to the cosine distance metric
- as a backup strategy, use the 1st sense of the word (e.g. <code>Lemma('run.n.01.run')</code>) from WordNet. You can look it up using a built-in function from NLTK (e.g. <code>wn.lemmas('run')</code>).

Calculate WSD accuracy in percentage points on your test data. Report three numbers
- overall accuracy (proportion of times the ELMo method+WordNet backup results in the correct sense annotation)
- WordNet baseline accuracy: what if you always select the first WordNet sense, ignoring the ELMo embedding?
- accuracy of the ELMo method just for the instances in which ELMo strategy is applicable
- accuracy of the WordNet baseline just for the instances in which ELMo strategy is applicable

 **Do not delete the output of this cell in the submitted version**

In [63]:
from torch.nn.functional import cosine_similarity
# Your code here
batch_size=50
correct=[0,0,0,0]
num_all_lemmas=0

vecs=[]
for val in Train_embeddings.values():
    vecs.append(val) 
vecs=torch.stack(vecs,dim=0)

def acc_batch(sentences, embs, sentences_tag, batch_size):
    global num_all_lemmas
    global correct
    for i in range(batch_size):

        lemmas_list=get_lemmas(sentences_tag[i])

        for k in range(len(lemmas_list)):

            # if not lemmas_list[k]:
            if type(lemmas_list[k])!=nltk.corpus.reader.wordnet.Lemma:
                continue
            num_all_lemmas+=1

            if len(wn.lemmas(sentences[i][k]))>0:
                wn_lemma=wn.lemmas(sentences[i][k])[0]

            else:
                wn_lemma=None
            similarities = torch.nn.functional.cosine_similarity(embs[i][k].expand(vecs.size()), vecs)
            elmo_lemma=list(Train_embeddings.keys())[similarities.argmax()]

            if type(elmo_lemma)!=nltk.corpus.reader.wordnet.Lemma or (not wn_lemma):# or type(elmo_lemma)==str
                continue
            if elmo_lemma==lemmas_list[k] or wn_lemma==lemmas_list[k]:
                correct[0]+=1
            if wn_lemma==lemmas_list[k]:
                correct[1]+=1
            if elmo_lemma==lemmas_list[k]:
                correct[2]+=1
            if wn_lemma==lemmas_list[k] and elmo_lemma==lemmas_list[k]:
                correct[3]+=1


for bat in range(int(len(semcor_test)/batch_size)):
    embeddings=sentences_to_elmo(semcor_test[bat*batch_size:(bat+1)*batch_size])
    print("###batch No. ",bat+1,"###")
    acc_batch(semcor_test[bat*batch_size:(bat+1)*batch_size], embeddings['elmo_representations'][0], semcor_tag_test[bat*batch_size:(bat+1)*batch_size], batch_size)

accuracy=correct[0]/num_all_lemmas
baseline_accuracy=correct[1]/num_all_lemmas
elmo_accuracy=correct[2]/num_all_lemmas
baseline_on_elmo_data_accuracy=correct[3]/num_all_lemmas

print("correct:",correct)
print("num_all_lemmas:",num_all_lemmas)

print("Overall accuracy:", accuracy) 
print("WordNet baseline:", baseline_accuracy)
print("Accuracy in cases where ELMo method is used", elmo_accuracy)
print("Accuracy of the baseline in cases where ELMo method is applicable", baseline_on_elmo_data_accuracy)

###batch No.  1 ###
###batch No.  2 ###
###batch No.  3 ###
###batch No.  4 ###
###batch No.  5 ###
###batch No.  6 ###
###batch No.  7 ###
###batch No.  8 ###
###batch No.  9 ###
###batch No.  10 ###
###batch No.  11 ###
###batch No.  12 ###
###batch No.  13 ###
###batch No.  14 ###
###batch No.  15 ###
###batch No.  16 ###
###batch No.  17 ###
###batch No.  18 ###
###batch No.  19 ###
###batch No.  20 ###
correct: [6681, 5704, 5701, 4724]
num_all_lemmas: 9709
Overall accuracy: 0.6881244206406427
WordNet baseline: 0.5874961376042847
Accuracy in cases where ELMo method is used 0.5871871459470595
Accuracy of the baseline in cases where ELMo method is applicable 0.4865588629107014


If you reached this point, you were able to evaluate ELMo as a model of contextual semantic similarity of word usages. The idea behind the vector averaging is that a word when used in the same sense should have similar vector representations, while usages in distinct senses should have different vector representations.

Analyze the numbers above. What do they tell you?

**WRITE YOUR ANALYSIS HERE**

Compared with the wordNet baseline, using ELMo can generate more representative embeddings with higher accuracy. Due to memory limitations, we only trained half of the data set, so the accuracy should be higher when training based on the complete data set.


## The end
Congratulations! this is the end of Lab 4.